In [ ]:
!pip install transformers seqeval[gpu]

In [ ]:
!pip install sentencepiece

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cpu


In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
sentences = []
with open('/content/drive/MyDrive/data/hindi_ner/hi_train.conll', 'r', encoding='utf-8') as in_fp:
    local = []
    for line in in_fp.readlines():
        sent = line.rstrip('\n')
        if(len(sent) == 0):
            sentences.append(local)
            local = []
        else:
            local.append(sent)

In [ ]:
global_sentence = []
global_labels = []
for i in sentences:
    sentence = ''
    tags = ''
    for j in range(1,len(i)):
        arr = i[j].split('_ _ ')
        sentence += (arr[0] + ' ')
        tags += (arr[1] + ',')
    global_sentence.append(sentence)
    global_labels.append(tags[:-1])

In [ ]:
df = pd.DataFrame({"sentence":global_sentence,"label":global_labels})
df

,sentence,label
0,जियोर्जियोन ०१९। जेपीजी | जियोर्जियोन आंध...,"O,O,O,O,O,B-CW,O,O,O,O,O"
1,इस क्षेत्र का सबसे प्रसिद्ध व्यंजन बिरिय...,"O,O,O,O,O,O,B-PROD,O,O,O,O,O,O,O,O,O,O,O,O,O,O..."
2,राजनीतिक टिप्पणीकार एंड्रयू सुलिवन ने मान...,"O,O,B-PER,I-PER,O,O,O,O,O,O,O,O,O,O,O,O"
3,लाइटिंग सर्किट से उपकरणों को जोड़ने के ...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-PROD,I-PROD,O,..."
4,तब से उन्हें तीन बार स्थानांतरित किया ग...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-LOC,O,O,O"
...,...,...
15295,चाओ फ्रया नदी मेक्सिको,"B-LOC,I-LOC,I-LOC,O"
15296,"चाय बोर्ड, भारत संगठन","B-GRP,I-GRP,I-GRP,O"
15297,कंप्यूटर अंकुरण,"O,B-PROD"
15298,एयर इंडिया रीजनल . की समीक्षाएं,"B-CORP,I-CORP,I-CORP,O,O,O"


In [ ]:
tags_list = []
for index, item in enumerate(global_labels):
    tags_list.extend(global_labels[index].split(","))

In [ ]:
from collections import Counter

In [ ]:
unique_list = list(Counter(tags_list).keys())

In [ ]:
if '' in unique_list:
  unique_list.remove('')

In [ ]:
unique_list

['O',
 'B-CW',
 'B-PROD',
 'B-PER',
 'I-PER',
 'I-PROD',
 'B-LOC',
 'B-CORP',
 'I-CORP',
 'B-GRP',
 'I-GRP',
 'I-CW',
 'I-LOC']

In [ ]:
labels_to_ids = {k: v for v, k in enumerate(unique_list)}
ids_to_labels = {v: k for v, k in enumerate(unique_list)}
ids_to_labels

{0: 'O',
 1: 'B-CW',
 2: 'B-PROD',
 3: 'B-PER',
 4: 'I-PER',
 5: 'I-PROD',
 6: 'B-LOC',
 7: 'B-CORP',
 8: 'I-CORP',
 9: 'B-GRP',
 10: 'I-GRP',
 11: 'I-CW',
 12: 'I-LOC'}

In [ ]:
labels_to_ids

{'B-CORP': 7,
 'B-CW': 1,
 'B-GRP': 9,
 'B-LOC': 6,
 'B-PER': 3,
 'B-PROD': 2,
 'I-CORP': 8,
 'I-CW': 11,
 'I-GRP': 10,
 'I-LOC': 12,
 'I-PER': 4,
 'I-PROD': 5,
 'O': 0}

In [ ]:
df.shape

(15300, 2)

In [ ]:
sentences = []
with open('/content/drive/MyDrive/data/hindi_ner/hi_dev.conll', 'r', encoding='utf-8') as in_fp:
    local = []
    for line in in_fp.readlines():
        sent = line.rstrip('\n')
        if(len(sent) == 0):
            sentences.append(local)
            local = []
        else:
            local.append(sent)

In [ ]:
global_sentence = []
global_labels = []
for i in sentences:
    sentence = ''
    tags = ''
    for j in range(1,len(i)):
        arr = i[j].split('_ _ ')
        sentence += (arr[0] + ' ')
        tags += (arr[1] + ',')
    global_sentence.append(sentence)
    global_labels.append(tags[:-1])

In [ ]:
test_df = pd.DataFrame({"sentence":global_sentence,"label":global_labels})
test_df

,sentence,label
0,"१४९२ में एक चार्टर के आधार पर, उसके पि...","O,O,O,O,O,O,O,O,O,O,O,B-LOC,O,O,O,O,O,O,O"
1,इस विश्वविद्यालय को ओडिशा के नॉलेज हब म...,"O,O,O,B-LOC,O,O,O,O,O,O,O,O,O"
2,"इस तरह का पहला बेड़ा, आर12 , १९४८ में ...","O,O,O,O,O,B-PROD,O,O,O,O,O,O,O,O"
3,"इस बिंदु पर, वायरल संजीन को अज्ञात तंत्...","O,O,O,O,B-PROD,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O..."
4,"ग्वाटेमाला में, कमजोर तूफान ने अत्यधिक व...","B-LOC,O,O,O,O,O,O,O,O,O,O,O"
...,...,...
795,"यह नए वाहनों, इस्तेमाल किए गए वाहनों और...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-CORP,I-CORP,O,O,..."
796,यह अपने प्रवक्ता के रूप में व्हूपी गोल्...,"O,O,O,O,O,O,B-PER,I-PER,O,O,O,O,O"
797,कभी-कभी नारियल के दूध की मिठास के आधार ...,"O,O,O,O,O,O,O,O,O,O,O,B-PROD,I-PROD,O,O,O,O,O,O"
798,"बपतिस्मात्मक फ़ॉन्ट , जिसमें करूबों से त...","B-PROD,I-PROD,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."


In [ ]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 2
LEARNING_RATE = 3*1e-03
MAX_GRAD_NORM = 10

In [ ]:
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
from transformers import AutoModel, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('ai4bharat/indic-bert')

In [ ]:
class dataset(Dataset):
  def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

  def __getitem__(self, index):
        sentence = self.data.sentence[index].strip()
        word_labels = self.data.label[index].split(",") 

        encoding = self.tokenizer(sentence,
                             return_offsets_mapping=True, 
                             padding='max_length', 
                             truncation=True, 
                             max_length=self.max_len)
        
        labels = [labels_to_ids[label] for label in word_labels]

        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.as_tensor(labels[0])
        
        return item

  def __len__(self):
        return self.len

In [ ]:
train_size = 0.85

train_dataset = df.sample(frac=train_size,random_state=200)
valid_dataset = df.drop(train_dataset.index).reset_index(drop=True)


train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))

print("VALIDATION Dataset: {}".format(valid_dataset.shape))

training_set = dataset(train_dataset, tokenizer, MAX_LEN)

valid_set = dataset(valid_dataset, tokenizer, MAX_LEN)

FULL Dataset: (15300, 2)
TRAIN Dataset: (13005, 2)
VALIDATION Dataset: (2295, 2)


In [ ]:
test_df = test_df.reset_index(drop=True)
testing_set = dataset(test_df,tokenizer, MAX_LEN)

In [ ]:
training_set[0]

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]),
 'input_ids': tensor([    2,   114,  3642, 12917,   507, 23316,   367, 29631,  4384,   293,
         57631,  9327,   229, 32721,  1883, 79942, 42601,   507,  5650, 17636,
            15,     3,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)
valid_loader = DataLoader(valid_set, **test_params)

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cpu


In [ ]:
from torch import nn
from transformers import AutoModel

In [ ]:

class additional_nn(nn.Module):
    def __init__(self):
        super(additional_nn, self).__init__()

        self.auto = AutoModel.from_pretrained('ai4bharat/indic-bert')
        self.classifier =  nn.Sequential(nn.Linear(768, 384), nn.ReLU(),
                                         nn.Linear(384, 192),nn.ReLU(),nn.Dropout(0.20),
                                         nn.Linear(192, 96),nn.ReLU(),nn.Dropout(0.20),
                                         nn.Linear(96,48),nn.ReLU(),nn.Dropout(0.10),
                                         nn.Linear(48,24),nn.ReLU(),nn.Dropout(0.05),
                                         nn.Linear(24,13))
        
    def forward(self,ids,attention_mask):
      output = self.auto(input_ids=ids,attention_mask=attention_mask)
      hidden_stat = output[0][:, 0, :]
      logits = self.classifier(hidden_stat)
      return logits

    def trigger_no_bert(self):
        for p in self.auto.parameters():
            p.requires_grad = False

    def feed_to_device(self,device):
        self.auto.to(device)

In [ ]:
loss_function = nn.CrossEntropyLoss()

In [ ]:
new_model = additional_nn()
new_model.trigger_no_bert()
new_model.feed_to_device(device)

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertModel: ['predictions.decoder.weight', 'sop_classifier.classifier.bias', 'sop_classifier.classifier.weight', 'predictions.LayerNorm.bias', 'predictions.dense.weight', 'predictions.decoder.bias', 'predictions.LayerNorm.weight', 'predictions.dense.bias', 'predictions.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
new_model

additional_nn(
  (auto): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(200000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
       

In [ ]:
def train(epochs):

  tr_loss, tr_accuracy = 0, 0
  nb_tr_examples, nb_tr_steps = 0, 0
  tr_preds, tr_labels = [], []
  
  optimizer = torch.optim.Adam(params=new_model.parameters(), lr=LEARNING_RATE)
  new_model.train()
  for idx, batch in enumerate(training_loader):
    ids = batch['input_ids'].to(device, dtype = torch.long)
    mask = batch['attention_mask'].to(device, dtype = torch.long)
    labels = batch['labels'].to(device, dtype = torch.long)
    
    logits = new_model(ids,mask)
    loss = loss_function(logits, labels)
    tr_loss += loss.item()
  
    nb_tr_steps += 1
    nb_tr_examples += labels.size(0)
    
    if idx % 100==0:
        loss_step = tr_loss/nb_tr_steps
        print(f"Training loss per 100 training steps: {loss_step}")

    flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
    active_logits = logits.view(-1,13) # shape (batch_size * seq_len, num_labels)
    flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len)

    active_accuracy = labels.view(-1) != -100

    labels = torch.masked_select(flattened_targets, active_accuracy)
    predictions = torch.masked_select(flattened_predictions, active_accuracy)
    tr_labels.extend(labels)
    tr_preds.extend(predictions)

    tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
    tr_accuracy += tmp_tr_accuracy
  
    torch.nn.utils.clip_grad_norm_(
        parameters=new_model.parameters(), max_norm=MAX_GRAD_NORM
    )
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  epoch_loss = tr_loss / nb_tr_steps
  tr_accuracy= tr_accuracy / nb_tr_steps
  print(f"Training loss epoch: {epoch_loss}")
  print(f"Training accuracy epoch: {tr_accuracy}")

In [ ]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training loss per 100 training steps: 2.593238115310669
Training loss per 100 training steps: 1.0096768950398016
Training loss per 100 training steps: 0.9471709672454282
Training loss per 100 training steps: 0.9368551983761035
Training loss per 100 training steps: 0.8926938281596599
Training loss per 100 training steps: 0.8779587533488542
Training loss per 100 training steps: 0.8674344247402298
Training loss per 100 training steps: 0.8546844210008029
Training loss per 100 training steps: 0.8496953718313079
Training loss per 100 training steps: 0.8496262861289638
Training loss per 100 training steps: 0.8495418255112239
Training loss per 100 training steps: 0.8396045398068607
Training loss per 100 training steps: 0.8385532449924703
Training loss per 100 training steps: 0.8328988993816725
Training loss per 100 training steps: 0.8261947624138067
Training loss per 100 training steps: 0.8228587359003847
Training loss per 100 training steps: 0.8192421814123125
Training loss 

In [ ]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            
            loss, eval_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
            
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              

            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    labels = [ids_to_labels[id.item()] for id in eval_labels]
    predictions = [ids_to_labels[id.item()] for id in eval_preds]
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

In [ ]:
labels, predictions = valid(new_model, testing_loader)

In [ ]:
from seqeval.metrics import classification_report

print(classification_report(labels, predictions))